---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [3]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head()

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
dtype: object

In [9]:
def date_sorter():    
    import re
    from functools import reduce
    
    re1 = r'(?:(?P<Month1>\d{1,2})[/-])?(?:(?P<Date1>\d{1,2})[/-])?(?:(?P<Year1>(?:1|2)\d{3}))'
    re2 = r'(?:(?P<Month2>0?[1-9]|1[0-2])[/-])(?:(?P<Date2>0?[1-9]|1\d|2\d|3[01])[/-])(?:(?P<Year2>\d{2}))'
    re3 = r'(?:(?P<Month3>0?[1-9]|1[0-2])/)(?:(?P<Year3>\d{2} ))'
    re4 = r'(?:(?P<Date4>\d{1,2})\s)?(?:(?P<Month4>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z\.]*[,]?\s)(?:(?P<Year4>\d{4}))'
    re5 = r'(?:(?P<Month5>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z\.]*\s)(?:(?P<Date5>\d{1,2})[,]?\s)(?:(?P<Year5>\d{4}))'
    
    temp = df.str.extractall("|".join([re1, re2, re3, re4, re5]))
    
    #return temp[194:250]
        
    temp['Year'] = reduce(lambda x,y: x.combine_first(y), [temp['Year1'], temp['Year2'], temp['Year3'], temp['Year4'], temp['Year5']])
    temp['Month'] = reduce(lambda x,y: x.combine_first(y), [temp['Month1'], temp['Month2'], temp['Month3'], temp['Month4'], temp['Month5']])
    temp['Date'] = reduce(lambda x,y: x.combine_first(y), [temp['Date1'], temp['Date2'], temp['Date4'], temp['Date5']])
    temp = temp[['Year', 'Month', 'Date']]
    temp = temp.fillna(1)
    mapping = {'Jan':1, 'Sep':9, 'May':5, 'Jun':6, 'Oct':10, 'Nov':11, 'Feb':2, 'Mar':3, 'Aug':8, 'Dec':12, 'Apr':4, 'Jul':7}
    temp['Month'] = temp['Month'].replace(mapping, regex=True)
    temp = temp.apply(pd.to_numeric)
    temp['Year'] = [1900+x if x<100 else x for x in temp['Year']]
    #return temp[194:250]
    
    return pd.Series(temp.sort_values(['Year', 'Month', 'Date']).index.labels[0])
date_sorter()

,,Year,Month,Date
,match,,,
194,0,1990,4,11
195,0,2001,5,30
196,0,1994,2,18
197,0,1981,2,18
198,0,2013,10,11
199,0,1986,1,24
200,0,1978,7,26
201,0,1999,12,23
202,0,1989,5,15
